In [ ]:
# pip install scikit-image
# import skimage
# import skimage
# print(skimage.__version__)
# pip install --upgrade scikit-image


import important modules

In [ ]:
import pandas as pd
import numpy as np
import skimage
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.feature import local_binary_pattern
# from skimage.feature import greycomatrix, greycoprops

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

Load dataset

In [ ]:
df = pd.read_csv(r"E:\Desktop Content\Project_IITB\submission\train_labels.csv")

In [ ]:
df.drop(['score'], axis=1, inplace=True)

In [ ]:
df

In [ ]:
df = df.rename(columns={"has_oilpalm": "label"})
len(df)

In [ ]:
for i in range(len(df)):
    hold = df.iloc[i, 0]
    hold = hold[0:-8] + hold[-4::]
    df.iloc[i,0] = hold

In [ ]:
df

In [ ]:
df.drop(df.index[15057:], inplace=True)

In [ ]:
df

import os moduel use to open directory where our dataset is present

In [ ]:
import os
available_imgs_lst = os.listdir(r"E:\Desktop Content\Project_IITB\submission\train_images")

In [ ]:
df = df[df['image_id'].isin(available_imgs_lst)]
df = df.drop_duplicates(subset='image_id')

In [ ]:
df.reset_index(inplace=True)

In [ ]:
df.drop(['index'], axis=1, inplace = True)

In [ ]:
df.drop(['index'], axis=1, inplace = True)

In [ ]:
import pickle
import joblib
import sklearn

find accuracy score at different distance by using svm classifier

In [ ]:
# distance = 1
# distance=1
import os
import numpy as np
from skimage.io import imread
from skimage.color import rgb2gray
# from skimage.feature import local_binary_pattern, greycomatrix, greycoprops

data = []

# Iterate over each image in the dataframe
for index, row in df.iterrows():
    filename = os.path.join(r"E:\Desktop Content\Project_IITB\submission\train_images", row['image_id'])
    img = imread(filename)
    
    # Convert image to grayscale
    gray = rgb2gray(img)
    
    # Calculate LBP
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(gray, n_points, radius)
    lbp = lbp / np.max(lbp)
    gray_int = np.interp(lbp, (gray.min(), gray.max()), (0, 255)).astype(np.uint8)
    
    # Calculate GLCM
    glcm = skimage.feature.graycomatrix(gray_int, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
    
    # Extract GLCM features
    contrast = skimage.feature.graycoprops(glcm, 'contrast')
    homogeneity = skimage.feature.graycoprops(glcm, 'homogeneity')
    energy = skimage.feature.graycoprops(glcm, 'energy')
    correlation = skimage.feature.graycoprops(glcm, 'correlation')
    
    # Concatenate the features into a single feature vector
    glcm_features = np.concatenate((contrast.ravel(), homogeneity.ravel(), energy.ravel(), correlation.ravel()))
    
    # Append the feature vector to the data list
    data.append(glcm_features)

# Convert the list of feature vectors into a numpy array
data = np.array(data)
# time taken by this code to execute is 9 min 53.3 s



In [ ]:
data

In [ ]:
labels = df['label']

# Convert the list of feature vectors into a NumPy array
data = np.array(data)
labels = np.array(labels)
# Split the data into training and testing sets
print(type(data))
print(len(data))
print("For Distance = 1")
print(type(labels))
print(len(labels))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

#X_train, X_test, y_train, y_test = train_test_split(0.2, 0.8, 42, data, labels)


# Train an SVM classifier on the training data
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

#Saving the weights
with open('model_pickle_11', 'wb') as f:
  pickle.dump(clf,f)

joblib.dump(clf, "model_joblib_11.pkl")

scikit_ver = sklearn.__version__
joblib.dump(clf, "model_{version}.pk1".format(version = scikit_ver))

# Use the trained classifier to predict labels for the testing data
y_pred = clf.predict(X_test)


# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

In [ ]:
# load and preprocess new image
image = imread(r"E:\Desktop Content\Project_IITB\submission\img_00109.jpg")
gray = rgb2gray(image)

radius = 1
n_points = 8 * radius
lbp = local_binary_pattern(gray, n_points, radius)

# Normalize the LBP image to the range [0, 1]
lbp = lbp / np.max(lbp)
gray_int = np.interp(lbp, (gray.min(), gray.max()), (0, 255)).astype(np.uint8)

#Calculating the GLCM
glcm = skimage.feature.graycomatrix(gray_int, distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)

# Extract GLCM features from the matrix
contrast = skimage.feature.graycoprops(glcm, 'contrast')
homogeneity = skimage.feature.graycoprops(glcm, 'homogeneity')
energy = skimage.feature.graycoprops(glcm, 'energy')
correlation = skimage.feature.graycoprops(glcm, 'correlation')

# Concatenate the features into a single feature vector
obs = np.concatenate((contrast.ravel(), homogeneity.ravel(), energy.ravel(), correlation.ravel()))

In [ ]:
# The trained classifier is saved to a file using both pickle and joblib modules.#
import pickle
with open(r"E:\Desktop Content\Project_IITB\submission\model_pickle_1 - Copy", 'rb') as f:
  mod = pickle.load(f)

In [ ]:
# The classifier is used to predict labels for the testing data using predict.

mod.predict([obs])

In [ ]:
print(obs)

In [ ]:
print(obs[1])

In [ ]:
import joblib

classifier = joblib.load(r"E:\Desktop Content\Project_IITB\submission\model_pickle_11")
# classifier=joblib.load(r"E:\Desktop Content\Project_IITB\submission\model_pickle_1 - Copy")
classifier.predict([obs])

In [ ]:
# The saved classifier is loaded using joblib.load.

classifier_ver = joblib.load(r"E:\Desktop Content\Project_IITB\submission\model_pickle_11")
classifier_ver.predict([obs])

In [ ]:
acc_1 = 0.9468623481781376
acc_dist.append(acc_1)

In [ ]:

# distance=2
data = []
for i in range(len(df)):
    filename = os.path.join("train_images", str(df.iloc[i, 0]))
    img = imread(filename)
    gray = rgb2gray(img)
    
    # Convert the gray image to integer data type
    gray_int = (gray * 255).astype(np.uint8)
    
    radius = 1
    n_points = 8 * radius
    lbp = local_binary_pattern(gray_int, n_points, radius)

    # Normalize the LBP image to the range [0, 1]
    lbp = lbp / np.max(lbp)

    # Calculating the GLCM
    glcm = skimage.feature.graycomatrix(gray_int, distances=[2], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)

    # Extract GLCM features from the matrix
    contrast = skimage.feature.graycoprops(glcm, 'contrast')
    homogeneity = skimage.feature.graycoprops(glcm, 'homogeneity')
    energy = skimage.feature.graycoprops(glcm, 'energy')
    correlation = skimage.feature.graycoprops(glcm, 'correlation')

    # Concatenate the features into a single feature vector
    glcm_features = np.concatenate((contrast.ravel(), homogeneity.ravel(), energy.ravel(), correlation.ravel()))
        
    data.append(glcm_features)


In [ ]:
labels = df['label']

# Convert the list of feature vectors into a NumPy array
data = np.array(data)
labels = np.array(labels)
# Split the data into training and testing sets
print(type(data))
print(len(data))
print("For Distance = 2")
print(type(labels))
print(len(labels))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

#X_train, X_test, y_train, y_test = train_test_split(0.2, 0.8, 42, data, labels)


# Train an SVM classifier on the training data
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

#Saving the weights
with open('model_pickle_12', 'wb') as f:
  pickle.dump(clf,f)

joblib.dump(clf, "model_joblib_12.pkl")

scikit_ver = sklearn.__version__
joblib.dump(clf, "model_{version}_dist2.pkl".format(version = scikit_ver))

# Use the trained classifier to predict labels for the testing data
y_pred = clf.predict(X_test)

# Use the trained classifier to predict labels for the testing data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


In [ ]:
# extract features for one sample
image = imread(r"E:\Desktop Content\Project_IITB\submission\img_00109.jpg")
gray = rgb2gray(image)

# Convert the gray image to integer data type
gray_int = (gray * 255).astype(np.uint8)

radius = 1
n_points = 8 * radius
lbp = local_binary_pattern(gray_int, n_points, radius)

# Normalize the LBP image to the range [0, 1]
lbp = lbp / np.max(lbp)

# Calculating the GLCM
glcm = skimage.feature.graycomatrix(gray_int, distances=[2], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)

# Extract GLCM features from the matrix
contrast = skimage.feature.graycoprops(glcm, 'contrast')
homogeneity = skimage.feature.graycoprops(glcm, 'homogeneity')
energy = skimage.feature.graycoprops(glcm, 'energy')
correlation = skimage.feature.graycoprops(glcm, 'correlation')

# Concatenate the features into a single feature vector
glcm_features = np.concatenate((contrast.ravel(), homogeneity.ravel(), energy.ravel(), correlation.ravel()))

# Append the feature vector to the data list
data.append(glcm_features)


In [ ]:
with open('model_pickle_12', 'rb') as f:
  mod = pickle.load(f)

In [ ]:
mod.predict([obs])

In [ ]:
classifier = joblib.load('model_joblib_12.pkl')
classifier.predict([obs])

In [ ]:
acc_2 = 0.9453441295546559
acc_dist.append(acc_2)


In [ ]:
# distance=4
data = []
for i in range(len(df)):
    filename = os.path.join("train_images", str(df.iloc[i, 0]))
    img = imread(filename)
    
    # Convert the image to grayscale
    gray = rgb2gray(img)
    
    # Convert the gray image to integer data type
    gray_int = (gray * 255).astype(np.uint8)
    
    radius = 1
    n_points = 8 * radius
    
    # Calculate LBP
    lbp = local_binary_pattern(gray_int, n_points, radius, method='uniform')
    
    # Normalize the LBP image to the range [0, 1]
    lbp = lbp / np.max(lbp)
    
    # Calculate GLCM
    glcm = skimage.feature.graycomatrix(gray_int, distances=[4], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
    
    # Extract GLCM features from the matrix
    contrast = skimage.feature.graycoprops(glcm, 'contrast')
    homogeneity = skimage.feature.graycoprops(glcm, 'homogeneity')
    energy = skimage.feature.graycoprops(glcm, 'energy')
    correlation = skimage.feature.graycoprops(glcm, 'correlation')
    
    # Concatenate the features into a single feature vector
    glcm_features = np.concatenate((contrast.ravel(), homogeneity.ravel(), energy.ravel(), correlation.ravel()))
        
    data.append(glcm_features)


In [ ]:
labels = df['label']

# Convert the list of feature vectors into a NumPy array
data = np.array(data)
labels = np.array(labels)
# Split the data into training and testing sets
print(type(data))
print(len(data))
print("For Distance = 4")
print(type(labels))
print(len(labels))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

#X_train, X_test, y_train, y_test = train_test_split(0.2, 0.8, 42, data, labels)


# Train an SVM classifier on the training data
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

#Saving the weights
with open('model_pickle_13', 'wb') as f:
  pickle.dump(clf,f)

joblib.dump(clf, "model_joblib_13.pkl")

scikit_ver = sklearn.__version__
joblib.dump(clf, "model_{version}_dist4.pkl".format(version = scikit_ver))


# Use the trained classifier to predict labels for the testing data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy (dist = 4): {accuracy}")

In [ ]:
image = imread(r"E:\Desktop Content\Project_IITB\submission\img_00109.jpg")
gray = rgb2gray(image)

radius = 1
n_points = 8 * radius
lbp = local_binary_pattern(gray, n_points, radius)

# Normalize the LBP image to the range [0, 1]
lbp = lbp / np.max(lbp)
gray_int = np.interp(lbp, (gray.min(), gray.max()), (0, 255)).astype(np.uint8)

#Calculating the GLCM
glcm = skimage.feature.graycomatrix(gray_int, distances=[4], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)

# Extract GLCM features from the matrix
contrast = skimage.feature.graycoprops(glcm, 'contrast')
homogeneity = skimage.feature.graycoprops(glcm, 'homogeneity')
energy = skimage.feature.graycoprops(glcm, 'energy')
correlation = skimage.feature.graycoprops(glcm, 'correlation')

# Concatenate the features into a single feature vector
obs = np.concatenate((contrast.ravel(), homogeneity.ravel(), energy.ravel(), correlation.ravel()))

In [ ]:
with open('model_pickle_13', 'rb') as f:
  mod = pickle.load(f)

In [ ]:
mod.predict([obs])

In [ ]:
print(obs[0])


In [ ]:
classifier = joblib.load('model_joblib_13.pkl')
classifier.predict([obs])

In [ ]:
acc_4 = 0.9473684210526315
acc_dist.append(acc_4)

In [ ]:
# distance=8
data = []
for i in range(len(df)):
    filename = os.path.join("train_images", str(df.iloc[i, 0]))
    img = imread(filename)
    
    # Convert the image to grayscale
    gray = rgb2gray(img)
    
    # Convert the gray image to integer data type
    gray_int = (gray * 255).astype(np.uint8)
    
    radius = 1
    n_points = 8 * radius
    
    # Calculate LBP
    lbp = local_binary_pattern(gray_int, n_points, radius, method='uniform')
    
    # Normalize the LBP image to the range [0, 1]
    lbp = lbp / np.max(lbp)
    
    # Calculate GLCM
    glcm = skimage.feature.graycomatrix(gray_int, distances=[8], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)
    
    # Extract GLCM features from the matrix
    contrast = skimage.feature.graycoprops(glcm, 'contrast')
    homogeneity = skimage.feature.graycoprops(glcm, 'homogeneity')
    energy = skimage.feature.graycoprops(glcm, 'energy')
    correlation = skimage.feature.graycoprops(glcm, 'correlation')
    
    # Concatenate the features into a single feature vector
    glcm_features = np.concatenate((contrast.ravel(), homogeneity.ravel(), energy.ravel(), correlation.ravel()))
        
    data.append(glcm_features)


In [ ]:
labels = df['label']

# Convert the list of feature vectors into a NumPy array
data = np.array(data)
labels = np.array(labels)
# Split the data into training and testing sets
print(type(data))
print(len(data))
print("For Distance = 8")
print(type(labels))
print(len(labels))
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

#X_train, X_test, y_train, y_test = train_test_split(0.2, 0.8, 42, data, labels)


# Train an SVM classifier on the training data
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

#Saving the weights
with open('model_pickle_14', 'wb') as f:
  pickle.dump(clf,f)

joblib.dump(clf, "model_joblib_14.pkl")

scikit_ver = sklearn.__version__
joblib.dump(clf, "model_{version}_dist8.pkl".format(version = scikit_ver))


# Use the trained classifier to predict labels for the testing data
y_pred = clf.predict(X_test)

# Calculate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy (dist = 8): {accuracy}")

In [ ]:
image = imread(r"E:\Desktop Content\Project_IITB\submission\img_00109.jpg")
gray = rgb2gray(image)

radius = 1
n_points = 8 * radius
lbp = local_binary_pattern(gray, n_points, radius)

# Normalize the LBP image to the range [0, 1]
lbp = lbp / np.max(lbp)
gray_int = np.interp(lbp, (gray.min(), gray.max()), (0, 255)).astype(np.uint8)

#Calculating the GLCM
glcm = skimage.feature.graycomatrix(gray_int, distances=[8], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=256, symmetric=True, normed=True)

# Extract GLCM features from the matrix
contrast = skimage.feature.graycoprops(glcm, 'contrast')
homogeneity = skimage.feature.graycoprops(glcm, 'homogeneity')
energy = skimage.feature.graycoprops(glcm, 'energy')
correlation = skimage.feature.graycoprops(glcm, 'correlation')

# Concatenate the features into a single feature vector
obs = np.concatenate((contrast.ravel(), homogeneity.ravel(), energy.ravel(), correlation.ravel()))

In [ ]:
with open('model_pickle_14', 'rb') as f:
  mod = pickle.load(f)

In [ ]:
mod.predict([obs])

In [ ]:
classifier = joblib.load('model_joblib_14.pkl')
classifier.predict([obs])

In [ ]:
acc_8 = 0.9438259109311741
acc_dist.append(acc_8)

In [ ]:
print(acc_dist)

# visual analysis

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Example data
distance = [1, 2, 4, 8]
acc_dist = [0.85, 0.92, 0.95, 0.97]

# Create scatter plot
sns.set_theme()
plt.figure(figsize=(8, 6))  # Adjust the figure size if needed
sns.scatterplot(x=distance, y=acc_dist, sizes=(20, 200), color='red')

# Add labels and title
plt.xlabel('Distance')
plt.ylabel('Accuracy')
plt.title('Accuracy vs. Distance')

# Display the plot
plt.show()


In [ ]:
sns.set_theme()
sns.lineplot(
    x=distance, y=acc_dist, sizes=(20, 200))

In [ ]:
sns.set_context("poster")
sns.set_theme()
sns.lineplot(
    x=distance, y=acc_dist, sizes=(20, 200))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline
x = np.array(distance)
y = np.array(acc_dist)
  
xnew = np.linspace(x.min(), x.max(), 300)
  
gfg = make_interp_spline(x, y, k=3)
  
y_new = gfg(xnew)
  
plt.plot(xnew, y_new, color = 'chartreuse')
plt.plot(x, y, color = 'blue')
plt.scatter(x, y, color = 'black')
plt.xlabel("Distance")
plt.ylabel(" Classification Accuracy")
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline
x = np.array(distance)
y = np.array(acc_dist)
  
xnew = np.linspace(x.min(), x.max(), 300)
  
gfg = make_interp_spline(x, y, k=3)
  
y_new = gfg(xnew)
  
plt.plot(xnew, y_new, color = 'chartreuse', linestyle = 'dotted')
plt.plot(x, y, color = 'blue')
plt.scatter(x, y, color = 'black')
plt.xlabel("Distance")
plt.ylabel(" Classification Accuracy")
plt.show()

In [ ]:
plt.scatter(x, y, color = 'black')
plt.plot(xnew, y_new, color = 'chartreuse')

plt.xlabel("Distance")
plt.ylabel(" Classification Accuracy")
plt.show()

In [ ]:
plt.scatter(x, y, color = 'black')
plt.plot(x, y, color = 'blue')
plt.xlabel("Distance")
plt.ylabel(" Classification Accuracy")
plt.show()